Video explicativo [https://youtu.be/vQf35U5exNM?si=2K7GdXsK3_yP9-21]

# Tema 6: Algoritmo de Shor

Curso IAC02.

Autor: Alejandro Mata Ali

En este notebook vamos a implementar las funciones necesarias para realizar un algoritmo de Shor. Será enfocado en la parte clásica, tanto para preparar el circuito cuántico como para procesar la salida del mismo.

Una explicación del algoritmo de Shor: https://hmong.es/wiki/Shor%27s_algorithm

## Librerías

In [ ]:
import numpy as np

---
#Algoritmo general


El algoritmo de Shor es un algoritmo que permite factorizar el número $N =p \cdot q$ en los números primos $p$ y $q$ en un tiempo polinómico mediante técnicas de computación cuántica.

El algoritmo procede de la siguiente manera:

### **Parte clásica**

* Elegir un número aleatorio $a$ entre $2$ y $N-1$.
* Calcular $mcd(a, N)$ (máximo común divisor).
Si $mcd(a, N) \neq 1$, este número es un factor no trivial de $N$ y hemos acabado.
* Si $mcd(a, N) = 1$, usamos la parte cuántica para encontrar r, el período de la función:
$f(x)=a^{x}{\bmod {N}}$.
* Este r es el entero positivo más pequeño r para el cual $f(x+r)=f(x)$.
* Si r es impar, volvemos al paso 1 (porque necesitamos su mitad).
* Si $a^{r/2} = -1({\bmod {N}})$, volvemos al paso 1.
Sino, ambos $mcd(a^{r/2}+1,N)$ y $mcd(a^{r/2}-1,N)$ son factores no triviales de N, los primos que queremos.

### **Parte Cuántica**
Hay 2 posibilidades que estudiaremos más adelante, pero conceptualmente se basa en estudiar los autovalores del operador $U_a$ que nos convierte $|x\rangle$ en $|(a\cdot x)mod(N)\rangle$. Este operador unitario introduce una fase en dichos estados, la cual podemos medir con métodos de quantum phase estimation y nos permite obtener la periodicidad de la función $f(y)=a^{y}{\bmod {N}}$.

La idea reside en que vamos a calcular con una superposición de estados que entrelazaremos con el autoestado.

Como vamos a estudiar $f(y)=a^{y}{\bmod {N}}$, haremos que el estado de entrada sea $|1\rangle$, ya que es el autoestado de este operador.

---
# Funciones de la parte clásica


## Crea binario

Función que crea el binario de un número con el número necesario de bits

In [ ]:
def numbin(a, n):
    '''Funcion que convierte el numero decimal a en uno binario con n bits.

    Parametros:
    a : entero que queremos pasar a binario.
    n : entero del numero de bits que queremos para el binario.
    '''
    a = bin(a)[2:]         #Pasamos el numero a binario
    return '0'*(n-len(a)) + a#Añadimos los ceros a la izquierda necesarios para que tenga el numero de bits.

## Inversión modular

Función que implementa a módulo inverso de N.

In [ ]:
def m_inv(a, N):
    '''Funcion que implementa a modulo inverso de N.'''
    return pow(a, -1, N)

## Creador de N

Función que genera el número N a partir de dos números primos de entrada p y q. Además, busca el $a$ que queremos para realizar el algoritmo. Podemos indicarle cuales son los $a$ que hemos encontrado en anteriores llamadas. Tenemos que verificar que el mcd(a, N) es 1, ya que sino tenemos la solución (y un límite de iteraciones).

In [ ]:
def primador(p, q, itmax = 100, used_a = []):
    '''Funcion que implementa el crear un numero N a partir de 2 primos
    y nos da un numero a valido para el algoritmo.
    itmax es el numero de iteraciones permitidas para buscar este a.'''
    N   = p * q
    ver = True; itera = 0      #Comprueba que el maximo comun divisor de a y N sea 1.
    while ver and itera<itmax:
        a  = np.random.randint(2, N)#Entero aleatorio entre 2 y N.
        cd = np.gcd(a, N)
        if cd == 1 and (a not in used_a): ver = False
        itera += 1#Contador de iteraciones.
    if ver == True: print('No se ha encontrado un a valido.')
    return a, N

Hacemos una prueba

In [ ]:
#Numeros primos para construir N.
p = 3
q = 7

#-------------------------------------
#Ejecutamos la funcion o los introducimos a mano.
a, N = primador(p, q)
print('a = ', a, ', N = ', N)
#Determinamos el numero de qbits n para el algoritmo (y añadimos uno extra)
n = 1 + int(np.floor(np.log2(N))); n +=1

a =  10 , N =  21


## Fracción irreducible

Esta función obtiene la fracción irreducible de un número. La entrada será la fracción num/dem, siendo num el numerador y dem el denominador.

In [ ]:
def reductor(num, den):
    '''Esta función implementa la obtención de
    la fraccion irreducible de un numero.
    El número de entrada va a ser una fraccion num/den.
    Ambos parametros son enteros.
    Se basa en dividir entre todos los valores posibles.'''
    for i in range(num, 0, -1):
        if num % i == 0 and den % i == 0:
            num /= i; den /= i
    return int(num), int(den)

## Procesador de la salida

Obtenemos la lista de resultados del algoritmo con los números obtenidos con una frecuencia superior a 0, estos resultados serán $y$. Por tanto, haremos una entrada serán las count, con la que haremos una lista Valores cuyos elementos serán las key de counts, ordenadas de mayor a menor frecuencia.

Obtenemos $r$ como el denominador de la fracción irreducible de $\frac{y}{2^n}$ y aplicamos que $p = mcd(a^{r/2} - 1, N)$ y $q = mcd(a^{r/2} + 1, N)$ y comprobamos.

In [ ]:
def procesador(counts, a, n, N):
    #Guardamos las claves y las frecuencias
    Keys = list(counts.keys())
    Frecuencias = [ counts[key] for key in Keys]
    #Ordenamos las claves segun su frecuencia
    Keys, Frecuencias = zip(*sorted(zip(Keys, Frecuencias)))
    #Convertimos las claves a entero
    Valores = [int(_, 2) for _ in Keys]

    for i in range(len(Valores)):#El 0 no contribuye.
        if Valores[i] != 0:
            r = reductor(Valores[i], 2**n)[0]#El denominador.
            if r % 2 == 0:
                if a**int(r/2) != -1 % N:
                    p_obt = np.gcd(a**int(r/2) - 1, N);   q_obt = np.gcd(a**int(r/2) + 1, N)
                    if N%p_obt == 0 and p_obt != N and p_obt != 1:#Si p es divisor
                        print('\nr = ', r, ', salida de cuentas: ', Valores[i], '. Puesto: ', i+1)
                        print('Los primos son: ', p_obt, ', ', N//p_obt)
                        return p_obt, N//p_obt
                        break
                    elif N%q_obt == 0 and q_obt != N and q_obt != 1:
                        print('\nr = ', r, ', salida de cuentas: ', Valores[i], '. Puesto: ', i+1)
                        print('Los primos son: ', q_obt, ', ', N//q_obt)
                        return q_obt, N//q_obt
                        break

    #Si no encontramos lo que debiamos
    return 1, 1